In [1]:
import os
import sys
import yaml
import numpy as np 
import pandas as pd
import io
import re
import csv
import subprocess
import requests
from snakemake.exceptions import print_exception, WorkflowError

relative_dir = "/vortexfs1/omics/alexander/akrinos/euknique" # ".." if we're running from the scripts or something 
with open(relative_dir + "/config.yaml", "r") as configfile:
    config = yaml.load(configfile)
    
relative_dir = config["homedir"]
    
INPUTDIR = config["inputdir"]
SUBDIRECTDIR = config["subdirectory"]
REFERENCEDIR = config["referencedir"]
DATADIR = config["datadir"]

samples_avail = pd.read_csv("../data/forNCBI_MMETSP.csv")
sample_names = list(samples_avail.SAMPLE_NAME)
sample_names = [curr.split("C")[0].split("_")[0] for curr in sample_names]
    
##### CREATE SUBDIRECTORY TABLE #####

# We want to get the location of all the files we're interested in using.
sampledirs = os.listdir(INPUTDIR)

subdirectory_table = pd.DataFrame({'Directory': [], \
                                   'File': [], \
                                   'Index': []})

for s in sampledirs:
    files = list(set([p.split("_R")[0] for p in os.listdir(os.path.join(INPUTDIR, s))]))
    indices = [p.split("_S")[0] for p in files]
    indices = ["".join(i for i in index if not i.isdigit()) for index in indices]
    indices = [index.split("_") for index in indices]
    
    # this is only if you have the S000X in the file name
    indices = [[i for i in indices_short if i != 'S'] for indices_short in indices]   

    # AX3 indicates that we have an infected form of amoebaphyra
    if "AX3" in files[0].split("_"):
        indices[0].append("AB") 
    indices = ["_".join(sorted(list(set([i for i in index if i])))) for index in indices]
    thisset = pd.DataFrame({'Directory': [s] * len(files), \
                            'File': files, \
                            'Index': indices})
    subdirectory_table = subdirectory_table.append(thisset)

print(subdirectory_table)
subdirectory_table.to_csv(path_or_buf = os.path.join(relative_dir,SUBDIRECTDIR), sep = "\t")

  Directory                       File        Index
0     S0007  PN3_TP3_AX1_HT1_071119_S7  AX_HT_PN_TP
0     S0006            AX3_071119_2_S6        AB_AX
0     S0001            AX3_071119_1_S1        AB_AX
0     S0008          PN3_PN2_071119_S8           PN
0     S0003          TP1_TP2_071119_S3           TP
0     S0002          AX1_HT1_071119_S2        AX_HT
0     S0004          AX2_HT2_071119_S4        AX_HT
0     S0005          AX4_PN1_071119_S5        AX_PN


/vortexfs1/home/akrinos/.conda/envs/akrinos_env/lib/python3.6/site-packages/ipykernel_launcher.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  from ipykernel import kernelapp as app


In [2]:
print(config)

{'homedir': '/vortexfs1/omics/alexander/akrinos/euknique', 'inputdir': '/vortexfs1/omics/alexander/data/single-cell/2019-08-singlecell/WH_Pilot', 'outputdir': '/vortexfs1/omics/alexander/data/single-cell/alevin-WHPilot-12182019_2', 'scratch': '/vortexfs1/scratch/akrinos/drop-seq', 'indexdir': 'data/indices/', 'datadir': 'data/', 'referencedir': '/vortexfs1/omics/alexander/data/mmetsp/', 'configlist': 'alex,thaps,pn,het,amoeb', 'smallnamelist': 'AX,TP,PN,HT,AB', 'makesalmon': 1, 'maketg': 1, 'usereverse': 0, 'alex': 'Alexandrium-fundyense_MMETSP0347,Alexandrium-fundyense_MMETSP0196', 'ehux': 'Emiliania-huxleyi-374', 'thaps': 'Thalassiosira-sp._MMETSP1071', 'pn': 'Pseudo-nitzschia-pungens_MMETSP1060', 'het': 'Heterocapsa-triquestra_MMETSP0448', 'amoeb': 'Amoebophrya_MMETSP0795', 'subdirectory': 'data/subdirectory_table.tsv', 'date': '071119'}


In [3]:

# The org_list is the name of all the organisms we want to get references for
# This list has other entries in the configfile corresponding to the 
# reference we wish to use for that organism 
org_list = config["configlist"].split(",")
# The short_names list is the two-letter codes of these organisms
short_names = config["smallnamelist"].split(",")

# Now we'll build a list of repeated entries based on how many references
# we're using for each organism and where they are
list_orgs_short = []
MMETSP_names = []
list_orgs = []
for curr_num in range(len(org_list)):
    curr = org_list[curr_num]
    MMname_list = config[curr].split(",")
    MMname_curr = []
    orgnames_curr = []
    for mm in MMname_list:
        MMname_curr.append(mm.split("_")[1])
        orgnames_curr.append(mm.split("_")[0])
        
    list_orgs_short.append(short_names[curr_num])
    MMETSP_names.append(MMname_curr) # we want to add a list of entries if we have multiple transcriptomes/species
    list_orgs.append(orgnames_curr)
print(MMETSP_names)

[['MMETSP0347', 'MMETSP0196'], ['MMETSP1071'], ['MMETSP1060'], ['MMETSP0448'], ['MMETSP0795']]


In [4]:

# Make a directory for each species of interest and then save the FASTA files from Zenodo corresponding to each given species of interest
files_written = []
for gg in range(0,len(MMETSP_names)):
    file_names = []
    for f in MMETSP_names[gg]:
        # if the current sample is not in the MMETSP list
        print(f)
        if f not in sample_names:
            file_names.append(os.path.join(DATADIR,"Acatassembly.fasta"))
        else:
            file_names.append(os.path.join(REFERENCEDIR, f + "_clean.fasta"))
        
    species_dir_name = os.path.join(relative_dir, DATADIR) 
    for f in range(0, len(file_names)):
        curr_file = file_names[f]
        to_write = species_dir_name + list_orgs[gg][f].replace(" ", "") + "_" + MMETSP_names[gg][f] + "_nt.fasta"
        os.system("cp " + curr_file + " " + to_write)

    os.system("cat " + " ".join(file_names) + " > " + os.path.join(relative_dir, DATADIR, list_orgs_short[gg].replace(" ", "") + "_" + "combined" + "_nt.fasta"))
    to_write = os.path.join(relative_dir, DATADIR, list_orgs_short[gg].replace(" ", "") + "_" + "combined" + "_nt.fasta")
    print(to_write)
    files_written.append(to_write) # need to extend if using list option

MMETSP0347
MMETSP0196
/vortexfs1/omics/alexander/akrinos/euknique/data/AX_combined_nt.fasta
MMETSP1071
/vortexfs1/omics/alexander/akrinos/euknique/data/TP_combined_nt.fasta
MMETSP1060
/vortexfs1/omics/alexander/akrinos/euknique/data/PN_combined_nt.fasta
MMETSP0448
/vortexfs1/omics/alexander/akrinos/euknique/data/HT_combined_nt.fasta
MMETSP0795
/vortexfs1/omics/alexander/akrinos/euknique/data/AB_combined_nt.fasta


In [5]:
# Iterate through the combined files of transcriptomes we just wrote
counter = 0
for ff in range(0,len(files_written)):
    f = files_written[ff]
    g = MMETSP_names[ff]
    short_name = list_orgs_short[ff]
    
    file_loc = "../data/tgMap_" + short_name + ".tsv"#list_orgs[ff].replace(" ", "-") + "_" + g + ".tsv"
    os.system("touch " + file_loc)

    with open("../data/tgMap.tsv", 'wt') as tgMap_file:
        transcript_to_gene = csv.writer(tgMap_file, delimiter='\t')
        command = str("cat " + str(f) + " | grep \">\" | cut -f2 -d \">\" | cut -f1 -d \" \" > transcript_names.txt")
        #else:
        #    command = str("cat " + str(f) + " | grep \">\" | cut -f2 -d \">\" > transcript_names.txt")
        os.system(command)
        transcripts = open("transcript_names.txt", "r")
        for transcript in transcripts:
            #if g == "combined":
            #    genes = [transcript.replace("\n",""), list_orgs_short[ff] + "-" + transcript.split("TRINITY_")[1].split("_")[0]]
            #else:
            genes = [transcript.replace("\n",""), list_orgs_short[ff] + "-" + "DN" + transcript.split("|")[3].replace("\n", "")]
            counter = counter + 1
            transcript_to_gene.writerow(genes)
    
    os.system("mv " + os.path.join(relative_dir,"data/tgMap.tsv ") + file_loc)
    tgMap_file.close()

print("Done!")

Done!
